## Crop Inusrance Analysis

- Author: Eunkyoung Choi
- Version: Aug 9, 2022


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from matplotlib.lines import Line2D

# 1) Liability
- Total policies: yield + revenue
- yield_based loss insurance plan code : [1,4,12,20,31,55,84,86,87,90,92,96]
- revenue based loss insurance plan code: [2,3,5,6,16,17,21,22,23,25,32,35,36,42,44,45,47,51,61,63,73,76,88,89]
    - For ref: https://ftp.rma.usda.gov/pub/Publications/M13_Handbook/2021/Draft/P99F_1_Insurance_Plan_Code_List_Exhibit.pdf
    - For ref: https://legacy.rma.usda.gov/FTP/Publications/M13_Handbook/2005/approved/REC99FEXH.PDF

In [14]:
##### total liability:
header_list1 = ['year','state_code','state_ab','county_code','county_name','commodity_code','commodity_name','insurance_plan_code',
              'insurance_plan_name','coverage_category','delivery_type','coverage_level','policies_sold_count',
               'polocies_earning_premium_count','polocies_indemnified_count','units_earning_premium_count','units_indemnified_count',
               'quantity_type','net_reported_quantity','endorsed_companion_acres','liability','total_premium',
               'subsidy','state/private_subsidy','additional_subsidy','efa_premium_dis','indemnity_amount','loss_ratio']

## if not available under the folder, please go to the USDA-RMA Summary of Business webistes:
file_list0 = sorted(glob.glob("0_data_cause_of_loss/Total_liability_downloaded_2022_Aug/sobcov*.txt"))

cov_lev = pd.DataFrame()

for file in file_list0:
    print(file)
    temp1 = pd.read_csv(file,delimiter="|",names=header_list1)
    extract_list = ['year','state_code','state_ab','county_code','county_name','commodity_code','commodity_name','insurance_plan_code',
              'insurance_plan_name','coverage_category','coverage_level','endorsed_companion_acres','liability',
                   'indemnity_amount']
    cov_lev = cov_lev.append(temp1[extract_list],ignore_index=True)  


In [10]:
## Creating GEOID format:
cov_lev['state_code'] = cov_lev['state_code'].astype(str).str.zfill(2)
cov_lev['county_code'] = cov_lev['county_code'].astype(str).str.zfill(3)
cov_lev['GEOID'] = cov_lev['state_code'] + cov_lev['county_code']

In [11]:
## For no liability and indemnity data, make sure that liability is zero:
cov_lev.loc[((cov_lev['liability'].isnull()) & (cov_lev['indemnity_amount']==0)), 'liability'] = 0

In [12]:
## make our own insurance plan categories:
cov_lev.loc[cov_lev['insurance_plan_code'].isin([1,4,12,20,31,55,84,86,87,90,92,96]), 'insurance_cat'] = 'yield_based'
cov_lev.loc[cov_lev['insurance_plan_code'].isin([2,3,5,6,16,17,21,22,23,25,32,35,36,42,44,45,47,51,61,63,73,76,88,89]), 'insurance_cat'] ='revenue_based'
cov_lev['insurance_cat'].unique()

array(['yield_based', 'revenue_based', nan], dtype=object)

In [13]:
## aggregating geoid-crop-year-policy level
cov = cov_lev.groupby(['GEOID','year','commodity_code','insurance_cat','insurance_plan_code','insurance_plan_name'])['liability'].sum().reset_index()

# 2). Indeminty
- Data information: 1989~2020 (Could not find cause of loss code list from RMA. And earlir years (e.g., 1989-2000) only have cuase of loss code. Based on the cause of loss description, get the corresponding codes and categorize the earlier years).
- Unit: Ares (Later converted into Hectare)
- There are 2 sources:
    - this one provides yearly data, but there are missing data
          https://legacy.rma.usda.gov/data/col/indemnity/colindemnitiesonly_2011-2015.pdf
          https://legacy.rma.usda.gov/data/cause.html
    - This one provides monthly data. Used this one: https://www.rma.usda.gov/Information-Tools/Summary-of-Business/Cause-of-Loss

## 2.1) Opening data:

In [21]:
header_list = ['year','state_code','state_ab','county_code','county_name','commodity_code','commodity_name','insurance_plan_code',
              'insurance_plan_name','coverage_category','stage_code','cause_of_loss_code','Damage Cause','month','month_name',
              'year_again','policies_earning_premium','polocies_indemnified','net_endorsed_acres','net_planted_acres',
              'liability','total_premium','producer_paid_premium','subsidy','state/private_subsidy','additional_subsidy','efa_premium_dis','Determined Acres',
              'indemnity_amount','loss_ratio']


file_list = sorted(glob.glob("0_data_cause_of_loss/New_downloaded_2022_Apr/col*.txt"))

data = pd.DataFrame()

for file in file_list:
    print(file)
    temp1 = pd.read_csv(file,delimiter="|",names=header_list)
    extract_list = ['year','state_code','state_ab','county_code','county_name','insurance_plan_name','insurance_plan_code','commodity_code','commodity_name','cause_of_loss_code','Damage Cause',
                     'month','month_name','Determined Acres','net_planted_acres','indemnity_amount','liability']#,'total_premium','producer_paid_premium','loss_ratio','policies_earning_premium','polocies_indemnified','subsidy','state/private_subsidy','additional_subsidy','efa_premium_dis']
    data = data.append(temp1[extract_list],ignore_index=True)  



In [16]:
data['state_code'] = data['state_code'].astype(str).str.zfill(2)
data['county_code'] = data['county_code'].astype(str).str.zfill(3)
data['GEOID'] = data['state_code'] + data['county_code']

## 2.2) Quality Check
1. Loss of cause code, Damage cause: Unify string format.
2. Indemnity amount: Should be all positive. Remove negative indemnity amount
3. Commodity name: Unify into string format and upper case. lower and upper cases are mixed in orginal dataframe.

## 2.2.1) Fix cause of loss code format:

In [17]:
data['cause_of_loss_code'] = data['cause_of_loss_code'].replace('XX',9999)
print(data['cause_of_loss_code'].astype(int).astype(str).unique())
data['cause_of_loss_code'] = data['cause_of_loss_code'].astype(int).astype(str).str.zfill(2)
data['cause_of_loss_code'].nunique()

['10' '11' '31' '42' '71' '81' '64' '12' '0' '51' '21' '32' '82' '99' '61'
 '44' '93' '41' '92' '43' '13' '62' '96' '94' '95' '98' '90' '91' '72'
 '97' '22' '45' '14' '63' '23' '55' '2' '1' '66' '76' '80' '70' '67' '65'
 '74' '9' '9999' '15' '87' '85' '88']


51

## 2.2.2) Fix indemnity data:
- No negative!
- Exclude nan and zero values!

In [18]:
## set negative indemnity to zero since there is no loss for these
data.loc[data['indemnity_amount']<0,'indemnity_amount'] = 0
data.loc[data['indemnity_amount'] <0]

,year,state_code,state_ab,county_code,county_name,insurance_plan_name,insurance_plan_code,commodity_code,commodity_name,cause_of_loss_code,Damage Cause,month,month_name,Determined Acres,net_planted_acres,indemnity_amount,liability,GEOID


## 2.2.3) Fix commodity name and damage cause format:

In [19]:
# Remove any space after string:
data['Damage Cause'] = data['Damage Cause'].str.rstrip()
data['commodity_name'] = data['commodity_name'].str.strip()
data['commodity_name'] = data['commodity_name'].str.upper()

In [20]:
## fixing typo
data.loc[data['Damage Cause'] == 'Ice Flow', 'Damage Cause'] = 'Ice Floe'

## 2.3) Find unique Code and Damage:

In [22]:
## get unique code - damange cause list:
code_cause = data.loc[(~data['cause_of_loss_code'].isnull()) & (~data['Damage Cause'].isnull())][['cause_of_loss_code','Damage Cause']].drop_duplicates(ignore_index=True).copy()

In [23]:
code_cause['repeated_damage'] = code_cause.groupby(['cause_of_loss_code'])['Damage Cause'].transform('count')
code_cause['Damage_Cause'] = code_cause.groupby(['cause_of_loss_code'])['Damage Cause'].transform('sum')


In [24]:
code_cause_df = code_cause[['cause_of_loss_code','Damage_Cause']].drop_duplicates(ignore_index=True).sort_values(by='cause_of_loss_code')
print(code_cause_df.shape)


(51, 2)


# 3) Final cause of loss data:

In [25]:
new_data = pd.merge(data.drop(columns=['Damage Cause']),code_cause_df, on=['cause_of_loss_code'], how='outer', indicator=True)
new_data.to_parquet('2_formatted_cause_of_loss_all_1989_2020.gzip',compression='gzip')

In [26]:
new_data = new_data.rename(columns={'liability':'fit_liability'})
print(new_data.shape)
new_data.head()

(3779316, 19)


,year,state_code,state_ab,county_code,county_name,insurance_plan_name,insurance_plan_code,commodity_code,commodity_name,cause_of_loss_code,month,month_name,Determined Acres,net_planted_acres,indemnity_amount,fit_liability,GEOID,Damage_Cause,_merge
0,1989,01,AL,001,Autauga,NaN,90,21,COTTON,10,10,OCT,0.0,0.0,0.0,0.0,01001,Drought Deviation,both
1,1989,01,AL,045,Dale,NaN,90,21,COTTON,10,10,OCT,0.0,0.0,0.0,0.0,01045,Drought Deviation,both
2,1989,01,AL,057,Fayette,NaN,90,9999,ALL OTHER CROPS,10,10,OCT,0.0,0.0,0.0,0.0,01057,Drought Deviation,both
3,1989,01,AL,113,Russell,NaN,90,9999,ALL OTHER CROPS,10,10,OCT,0.0,0.0,0.0,0.0,01113,Drought Deviation,both
4,1989,08,CO,063,Kit Carson,NaN,90,11,WHEAT,10,10,OCT,0.0,0.0,0.0,0.0,08063,Drought Deviation,both


# 4) Filtering data:

## 4.1) Extracting four crops:

In [28]:
new_data_hazard = new_data.loc[new_data['commodity_name'].isin(['CORN','SOYBEANS','GRAIN SORGHUM','WHEAT'])].copy()
new_data_hazard[new_data_hazard.isna().any(axis=1)]

,year,state_code,state_ab,county_code,county_name,insurance_plan_name,insurance_plan_code,commodity_code,commodity_name,cause_of_loss_code,month,month_name,Determined Acres,net_planted_acres,indemnity_amount,fit_liability,GEOID,Damage_Cause,_merge
4,1989,08,CO,063,Kit Carson,NaN,90,11,WHEAT,10,10,OCT,0.00,0.0,0.0,0.0,08063,Drought Deviation,both
5,1989,13,GA,091,Dodge,NaN,90,81,SOYBEANS,10,10,OCT,0.00,0.0,0.0,0.0,13091,Drought Deviation,both
6,1989,13,GA,093,Dooly,NaN,90,81,SOYBEANS,10,10,OCT,0.00,0.0,0.0,0.0,13093,Drought Deviation,both
7,1989,13,GA,235,Pulaski,NaN,90,81,SOYBEANS,10,10,OCT,0.00,0.0,0.0,0.0,13235,Drought Deviation,both
8,1989,13,GA,271,Telfair,NaN,90,81,SOYBEANS,10,10,OCT,0.00,0.0,0.0,0.0,13271,Drought Deviation,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3768802,2010,46,SD,037,Day,NaN,25,81,SOYBEANS,09,8,AUG,35.29,0.0,243.0,4379.0,46037,Asian Soybean Rust,both
3768803,2010,51,VA,093,Isle of Wight,NaN,25,81,SOYBEANS,09,11,NOV,91.50,0.0,8197.0,19003.0,51093,Asian Soybean Rust,both
3768804,2010,51,VA,175,Southampton,NaN,44,81,SOYBEANS,09,9,SEP,75.50,0.0,13060.0,19023.0,51175,Asian Soybean Rust,both
3768805,2010,51,VA,800,Suffolk City,NaN,44,81,SOYBEANS,09,9,SEP,127.73,0.0,13375.0,37516.0,51800,Asian Soybean Rust,both


In [29]:
## filter hazard events:
new_data_hazard.loc[new_data_hazard['Damage_Cause'].isin(['Drought','Drought Deviation']), 'cause'] = 'Drought'
new_data_hazard.loc[new_data_hazard['Damage_Cause'].isin(['Heat']), 'cause'] = 'Heat'
new_data_hazard.loc[new_data_hazard['Damage_Cause'].isin(['Excess Moisture/Precipitation/Rain','Flood','Oxygen Depletion']), 'cause'] = 'Rain/Flood'
new_data_hazard.loc[new_data_hazard['Damage_Cause'].isin(['Mycotoxin (Aflatoxin)Mycotoxin','Plant Disease','Asian Soybean Rust']), 'cause'] = 'Disease'

new_data_hazard = new_data_hazard.loc[~new_data_hazard['cause'].isnull()]
new_data_hazard['Damage_Cause'].unique()

array(['Drought Deviation', 'Drought',
       'Excess Moisture/Precipitation/Rain', 'Plant Disease', 'Heat',
       'Flood', 'Mycotoxin (Aflatoxin)Mycotoxin', 'Asian Soybean Rust'],
      dtype=object)

## 4-2) Extracting data under relevant insurance plan:


In [30]:
new_data_hazard.loc[new_data_hazard['insurance_plan_code'].isin([1,4,12,20,31,55,84,86,87,90,92,96]), 'insurance_cat'] = 'yield_based'
new_data_hazard.loc[new_data_hazard['insurance_plan_code'].isin( [2,3,5,6,16,17,21,22,23,25,32,35,36,42,44,45,47,51,61,63,73,76,88,89] ), 'insurance_cat'] = 'revenue_based'
new_data_hazard = new_data_hazard.loc[~new_data_hazard['insurance_cat'].isnull()] #only one for wheat - GRIP-HRO
new_data_hazard

,year,state_code,state_ab,county_code,county_name,insurance_plan_name,insurance_plan_code,commodity_code,commodity_name,cause_of_loss_code,...,month_name,Determined Acres,net_planted_acres,indemnity_amount,fit_liability,GEOID,Damage_Cause,_merge,cause,insurance_cat
4,1989,08,CO,063,Kit Carson,NaN,90,11,WHEAT,10,...,OCT,0.0,0.0,0.0,0.0,08063,Drought Deviation,both,Drought,yield_based
5,1989,13,GA,091,Dodge,NaN,90,81,SOYBEANS,10,...,OCT,0.0,0.0,0.0,0.0,13091,Drought Deviation,both,Drought,yield_based
6,1989,13,GA,093,Dooly,NaN,90,81,SOYBEANS,10,...,OCT,0.0,0.0,0.0,0.0,13093,Drought Deviation,both,Drought,yield_based
7,1989,13,GA,235,Pulaski,NaN,90,81,SOYBEANS,10,...,OCT,0.0,0.0,0.0,0.0,13235,Drought Deviation,both,Drought,yield_based
8,1989,13,GA,271,Telfair,NaN,90,81,SOYBEANS,10,...,OCT,0.0,0.0,0.0,0.0,13271,Drought Deviation,both,Drought,yield_based
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3768897,2020,31,NE,173,Thurston,RP,2,81,SOYBEANS,09,...,AUG,72.7,0.0,4598.0,30066.0,31173,Asian Soybean Rust,both,Disease,revenue_based
3768898,2020,39,OH,039,Defiance,RP,2,81,SOYBEANS,09,...,OCT,63.3,0.0,4407.0,17182.0,39039,Asian Soybean Rust,both,Disease,revenue_based
3768899,2020,42,PA,037,Columbia,RP,2,81,SOYBEANS,09,...,AUG,1225.2,0.0,123763.0,371251.0,42037,Asian Soybean Rust,both,Disease,revenue_based
3768900,2020,42,PA,111,Somerset,RP,2,81,SOYBEANS,09,...,JUL,105.0,0.0,13677.0,29271.0,42111,Asian Soybean Rust,both,Disease,revenue_based


### 4-1-1) Total: yield+revenue-based liability:

In [31]:
## total yield + revenue aggregating to yearly value per cause:
df = new_data_hazard.groupby(['GEOID','year','commodity_code','commodity_name','cause'])[['indemnity_amount','fit_liability']].sum().reset_index()
df

,GEOID,year,commodity_code,commodity_name,cause,indemnity_amount,fit_liability
0,01001,1995,41,CORN,Drought,2882.00,4490.00
1,01001,1995,81,SOYBEANS,Drought,631.00,842.00
2,01001,1997,81,SOYBEANS,Drought,4700.00,5603.00
3,01001,1997,81,SOYBEANS,Heat,4965.00,7042.00
4,01001,1998,41,CORN,Drought,12266.00,15608.00
...,...,...,...,...,...,...,...
277357,56045,2012,11,WHEAT,Drought,12723.25,28546.25
277358,56045,2012,11,WHEAT,Heat,367.50,1494.75
277359,56045,2013,11,WHEAT,Drought,1947.00,5044.00
277360,56045,2016,11,WHEAT,Drought,20861.75,29795.70


In [32]:
print(cov.loc[cov['insurance_cat'].isnull()])
df = pd.merge(df,cov.loc[cov['commodity_code'].isin([41, 81, 11, 51])].groupby(['GEOID','year','commodity_code'])['liability'].sum().reset_index(),
              on=['GEOID','year','commodity_code'], how='outer', indicator=True)
print(df._merge.unique())
df = df[df['_merge'] == 'both'] #right_only shows the liabilities in years that any cause of loss for our interest hasn't happened

Empty DataFrame
Columns: [GEOID, year, commodity_code, insurance_cat, insurance_plan_code, insurance_plan_name, liability]
Index: []
['both', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']


In [33]:
df.to_parquet('3_total_policy_based_cause_of_loss_df_1989_2020_updated_sep2022.gzip', compression='gzip')

### 4-1-3) Cleaning and plot:

In [34]:
df = df.loc[df['_merge'] == 'both'].drop(columns=['_merge'])
df

,GEOID,year,commodity_code,commodity_name,cause,indemnity_amount,fit_liability,liability
0,01001,1995,41,CORN,Drought,2882.00,4490.00,39120.0
1,01001,1995,81,SOYBEANS,Drought,631.00,842.00,1119.0
2,01001,1997,81,SOYBEANS,Drought,4700.00,5603.00,28867.0
3,01001,1997,81,SOYBEANS,Heat,4965.00,7042.00,28867.0
4,01001,1998,41,CORN,Drought,12266.00,15608.00,41697.0
...,...,...,...,...,...,...,...,...
277357,56045,2012,11,WHEAT,Drought,12723.25,28546.25,103968.0
277358,56045,2012,11,WHEAT,Heat,367.50,1494.75,103968.0
277359,56045,2013,11,WHEAT,Drought,1947.00,5044.00,79748.0
277360,56045,2016,11,WHEAT,Drought,20861.75,29795.70,32430.0


# 5. Analysis:
## 5-1.cleaning:

In [35]:
df.loc[df['indemnity_amount'] > df['liability'], 'indemnity_amount'] =df.loc[df['indemnity_amount'] > df['liability']]['liability'].values
df.loc[df['indemnity_amount'] > df['liability']]

,GEOID,year,commodity_code,commodity_name,cause,indemnity_amount,fit_liability,liability


# 5-2) Focusing on crop and Some filtration for Wheat:

In [36]:
crop = 'WHEAT'
crop_name_list = df[df['commodity_name'].str.contains(crop)]['commodity_name'].unique()
df[df['commodity_name'].str.contains(crop)]['commodity_name'].unique()

array(['WHEAT'], dtype=object)

In [37]:
df_event = df.loc[ (df['commodity_name'].isin([crop]))].copy()
df_event['commodity_name'].unique()

array(['WHEAT'], dtype=object)

In [38]:
## check if indem and total_premium data are all available:
df_event.loc[(df_event['indemnity_amount'].isnull()) | (df_event['liability'].isnull())]

,GEOID,year,commodity_code,commodity_name,cause,indemnity_amount,fit_liability,liability


In [39]:
df_event['cause'].unique()

array(['Drought', 'Rain/Flood', 'Disease', 'Heat'], dtype=object)

### For wheat, identify which crop is more dominant in a given county between spring and winter wheat:
- based on maxium harvested area:

In [40]:
if crop == 'WHEAT':
    harvested = pd.read_csv('../0_data/3_spring_wheat/1_survey_field_crop_spring_wheat_total_yields/survey_winter_wheat_harvested_acres_1981_1999_USDA_downloaded_3July2022.csv', index_col=0)
    harvested2 = pd.read_csv('../0_data/3_spring_wheat/1_survey_field_crop_spring_wheat_total_yields/survey_winter_wheat_harvested_acres_2000_2020_USDA_downloaded_3July2022.csv', index_col=0)
    harvested = pd.concat([harvested, harvested2])
    harvested['crop'] = 'Winter Wheat'
    harvested3 = pd.read_csv('../0_data/3_spring_wheat/1_survey_field_crop_spring_wheat_total_yields/survey_spr_wheat_harvested_acres_1981_2020_USDA_downloaded_3July2022.csv', index_col=0)
    harvested3['crop'] = 'Spring Wheat'
    harvested = pd.concat([harvested, harvested3])
del harvested2, harvested3
harvested.head()

,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,County ANSI,...,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%),crop
Program,,,,,,,,,,,,,,,,,,,,,
SURVEY,1999,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,AUTAUGA,1.0,...,NaN,0,NaN,WHEAT,"WHEAT, WINTER - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"3,100",NaN,Winter Wheat
SURVEY,1999,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,DALLAS,47.0,...,NaN,0,NaN,WHEAT,"WHEAT, WINTER - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"1,100",NaN,Winter Wheat
SURVEY,1999,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,ELMORE,51.0,...,NaN,0,NaN,WHEAT,"WHEAT, WINTER - ACRES HARVESTED",TOTAL,NOT SPECIFIED,600,NaN,Winter Wheat
SURVEY,1999,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,LOWNDES,85.0,...,NaN,0,NaN,WHEAT,"WHEAT, WINTER - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"1,000",NaN,Winter Wheat
SURVEY,1999,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,MACON,87.0,...,NaN,0,NaN,WHEAT,"WHEAT, WINTER - ACRES HARVESTED",TOTAL,NOT SPECIFIED,300,NaN,Winter Wheat


In [41]:
## cleaning 
harvested['Value'] = harvested['Value'].str.replace(',','').astype(float)
harvested = harvested.loc[~harvested['Value'].isnull()]
harvested = harvested.loc[~harvested['County ANSI'].isnull()]
harvested['GEOID'] = harvested['State ANSI'].astype(int).astype(str).str.zfill(2) +harvested['County ANSI'].astype(int).astype(str).str.zfill(3)
harvested['harvested_hac'] = harvested['Value'] * 1/0.404686 #changing acre -> hac but does not matter actually
harvested = harvested.rename(columns={'Year':'year'})[['year','GEOID','Data Item','crop','Value','harvested_hac']]
print(harvested.groupby(['GEOID'])['year'].nunique().unique())
harvested.head()

[23 27 19  6 14 11 18 20 10  1  7 16 21 25  8 12 24 26 28 22 15 17  4  9
  5 13  2  3 34 31 35 37 36 30 33 38 32 29 39 40]


,year,GEOID,Data Item,crop,Value,harvested_hac
Program,,,,,,
SURVEY,1999,01001,"WHEAT, WINTER - ACRES HARVESTED",Winter Wheat,3100.0,7660.260053
SURVEY,1999,01047,"WHEAT, WINTER - ACRES HARVESTED",Winter Wheat,1100.0,2718.156793
SURVEY,1999,01051,"WHEAT, WINTER - ACRES HARVESTED",Winter Wheat,600.0,1482.630978
SURVEY,1999,01085,"WHEAT, WINTER - ACRES HARVESTED",Winter Wheat,1000.0,2471.051630
SURVEY,1999,01087,"WHEAT, WINTER - ACRES HARVESTED",Winter Wheat,300.0,741.315489


In [42]:
final_harvested =harvested.groupby(['GEOID','crop'])['harvested_hac'].mean().reset_index()
del harvested
final_harvested

,GEOID,crop,harvested_hac
0,01001,Winter Wheat,6575.146077
1,01003,Winter Wheat,34137.120668
2,01005,Winter Wheat,3693.571910
3,01007,Winter Wheat,5683.418749
4,01009,Winter Wheat,6442.384607
...,...,...,...
3190,56039,Spring Wheat,247.105163
3191,56043,Spring Wheat,314.497480
3192,56043,Winter Wheat,321.236712
3193,56045,Spring Wheat,2067.880048


In [43]:
#### dominant harvested area:
final_harvested = final_harvested.pivot_table(index=['GEOID'],columns=['crop'], values=['harvested_hac'])
final_harvested.columns =['_'.join(col) for col in final_harvested.columns.values]
final_harvested = final_harvested.reset_index()
final_harvested.loc[(final_harvested['harvested_hac_Winter Wheat'] >0) & (final_harvested['harvested_hac_Spring Wheat'].isnull()), 'major_crop'] = 'Winter_Wheat'
final_harvested.loc[(final_harvested['harvested_hac_Spring Wheat']>0) & (final_harvested['harvested_hac_Winter Wheat'].isnull()), 'major_crop'] = 'Spring_Wheat'
final_harvested.loc[final_harvested['major_crop'] == 'Winter_Wheat','frac_winter'] = 1
final_harvested.loc[final_harvested['major_crop'].isnull(), 'frac_winter'] = (final_harvested.loc[final_harvested['major_crop'].isnull()]['harvested_hac_Winter Wheat'] / 
                                                                     (final_harvested.loc[final_harvested['major_crop'].isnull()]['harvested_hac_Winter Wheat'] + final_harvested.loc[final_harvested['major_crop'].isnull()]['harvested_hac_Spring Wheat']))

In [44]:
#### MAX
final_harvested
final_harvested.loc[(final_harvested['frac_winter'] > 0.5) & (final_harvested['major_crop'].isnull()), 'major_crop'] = 'Winter_Wheat'
final_harvested.loc[(final_harvested['frac_winter'] <= 0.5) & (final_harvested['major_crop'].isnull()), 'major_crop'] = 'Spring_Wheat'
print(final_harvested.loc[final_harvested['major_crop'] == 'Winter_Wheat']['GEOID'].nunique(),final_harvested.loc[final_harvested['major_crop'] == 'Spring_Wheat']['GEOID'].nunique())
final_harvested.loc[final_harvested['major_crop'].isnull()]

2426 251


,GEOID,harvested_hac_Spring Wheat,harvested_hac_Winter Wheat,major_crop,frac_winter


In [45]:
## remove winter wheat counties:
df_event = pd.merge(df_event, final_harvested, on=['GEOID'], how='outer', indicator=True)
print(df_event._merge.unique())
print(df_event.loc[df_event['_merge'] == 'left_only']['GEOID'].nunique())
df_event = df_event.loc[(df_event['_merge'] != 'right_only') & (df_event['major_crop'] != 'Winter_Wheat')].drop(columns=['_merge'])
df_event.loc[df_event['major_crop'] == 'Spring_Wheat']['GEOID'].nunique()

['both', 'left_only', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']
19


212

In [46]:
df_event['major_crop'].unique()

array(['Spring_Wheat', nan], dtype=object)

In [1]:
# temp_df = final_harvested.loc[~final_harvested['major_crop'].isnull()][['GEOID','major_crop']].drop_duplicates()
# #df_event.loc[~df_event['major_crop'].isnull()][['GEOID','major_crop']].drop_duplicates()

# fig = px.choropleth(temp_df, geojson=counties, locations='GEOID', color= 'major_crop',
#                     #color_continuous_scale= px.colors.sequential.speed, 
#                     scope="usa",title='Wheat: Spring vs. Winter Wheat County')
# fig.show()
# fig.write_image('wheat_filter_winter_based_on_harvested_area_revenue_based.png')

## 5-3) Yield loss data:


In [47]:
crop = 'WHEAT'

In [48]:
## hazard events:

if crop == 'CORN': 
    crop_yld_name = 'maize_yld'
    print(crop)
    hazards = {'HighD+LowS':['bin1v6','bin2v6','bin3v6'], 
               'HighD+HighS':['bin4v6','bin5v6'], 
               'LowD+HighS':['bin6v1','bin5v1']}
elif crop == 'SOYBEANS': 
    crop_yld_name = 'soy_yld'
    print(crop)
    hazards = {'HighD+LowS':['bin1v6','bin2v6','bin3v6','bin1v5'],
               'HighD+HighS':['bin4v6','bin5v6'],
               'LowD+HighS':['bin6v1']}
elif crop == 'WHEAT': 
    crop_yld_name = 'spr_wheat_yld'
    print(crop)
    hazards = {'HighD+LowS':['bin1v6','bin2v6','bin3v6','bin1v5'],
               'HighD+HighS':['bin4v6','bin5v6','bin6v6'],
               'LowD+LowS':['bin1v1']}
elif crop == 'GRAIN SORGHUM':
    crop_yld_name = 'sorghum_yld'
    print(crop)
    hazards = {'HighD+LowS':['bin1v6','bin2v6','bin3v6'],
               'HighD+HighS':['bin4v6','bin5v6'],
               'LowD+HighS':['bin6v1'],
               'LowD+LowS':['bin1v1', 'bin2v1']}
n_hazards = len(hazards.keys())

WHEAT


In [50]:
## here crop_name_yld : actual yield with an unit of bu/acre and estimated yield loss (=percent_n_b*coef) is kg/ha
pthh= '../3_outputs/spring_wheat_both_rainfed_irrigated_hydroclimatic_hazard_exposure_coef_22Sep2022.gzip'
ds = pd.read_parquet(pthh)
ds.head()

,GEOID,year,rsm_vpd_bin,n_b,spr_wheat_yld,full_gs_days,percent_n_b,type,planted_hac,harvested_hac,_merge,coef
0,16009,1981.0,bin4v4,1.0,29.1,126.0,0.793651,rainfed,14462.516069,13977.228985,both,0.0
1,16009,1982.0,bin4v4,3.0,35.0,126.0,2.380952,rainfed,14462.516069,13977.228985,both,0.0
2,16009,1983.0,bin4v4,1.0,42.8,126.0,0.793651,rainfed,14462.516069,13977.228985,both,0.0
3,16009,1984.0,bin4v4,3.0,35.0,126.0,2.380952,rainfed,14462.516069,13977.228985,both,0.0
4,16009,1986.0,bin4v4,8.0,30.0,126.0,6.349206,rainfed,14462.516069,13977.228985,both,0.0


In [51]:
## only rainfed:
ds = ds.loc[ds['type']=='rainfed']

In [52]:
col_list = df_event['cause'].unique()
col_list

array(['Rain/Flood', 'Heat', 'Drought', 'Disease'], dtype=object)

## 5-4) Test all possibilities:

In [53]:
crop_name = 'Spring Wheat'

In [61]:
fig, ax = plt.subplots(figsize=(6*n_hazards,20), nrows=5, ncols=n_hazards, sharey=True, tight_layout=True)
cor_temp = pd.DataFrame()
for idx, hazard in enumerate(list(hazards.keys())):
    print(hazard)
    ## copy exposure dataframe
    new_ds = ds.copy() 
    ## cleaning
    new_ds.loc[~new_ds['rsm_vpd_bin'].isin(hazards[hazard]), 'percent_n_b'] =0
    new_ds.loc[~new_ds['rsm_vpd_bin'].isin(hazards[hazard]), 'n_b']=0
    new_ds['yield_loss'] = new_ds['percent_n_b'] * new_ds['coef']
    tot_full = new_ds[['GEOID','year','full_gs_days']].drop_duplicates()['full_gs_days'].sum()
    ## exposure calculation
    exp = new_ds['n_b'].sum() / tot_full * 100
    new_ds = new_ds.groupby(['GEOID','year'])[['percent_n_b','yield_loss']].sum().reset_index()
    
    
    if hazard == 'HighD+LowS':
        ccolor= 'tab:red'
    elif hazard == 'LowD+HighS':
        ccolor= 'tab:blue'
    elif hazard == 'LowD+LowS':
        ccolor= 'tab:green'
    else:
        ccolor='tab:purple'
        
    for idy, col in enumerate(col_list):
        print(new_ds.shape)
        temp_df = pd.merge(new_ds, df_event.loc[df_event['cause'] == col], on=['GEOID','year'], how='outer', indicator=True)
        temp_df = temp_df.loc[temp_df['_merge'] == 'both'] ## != right only leads to highly tailed distribution, making misleading slope and tau.
        #temp_df.loc[temp_df['_merge'] == 'left_only', 'indemnity_amount'] = 0
        print(temp_df[temp_df['liability'] < temp_df['indemnity_amount']])
        temp_df['loss_cost'] = temp_df['indemnity_amount'] / temp_df['liability'] * 100

        ### Sen's slope gets incorrect with any of zero values in pairs -> x=0 or/and y=0 provide negative slopes (misleading)
        ### So remove all zero values in paires:
        temp_df = temp_df.loc[(temp_df['loss_cost'] !=0)]
        temp_df = temp_df.loc[(~temp_df['loss_cost'].isnull())]
        temp_df = temp_df.loc[(temp_df['yield_loss'] !=0)]
        
        print(temp_df.loc[temp_df['loss_cost'] >100], temp_df.loc[temp_df['loss_cost'] <0])
        print(temp_df.shape)
        yy = temp_df['loss_cost']
        xx = temp_df['yield_loss'] * (1/-1000)
        mank = stats.theilslopes(y=yy, x=xx)
        tau, pval = stats.kendalltau(x=xx, y=yy)
        rho, s_pval = stats.spearmanr(xx, yy)
        r, r_pval = stats.pearsonr(xx, yy)
        ax[idy, idx].scatter(x=xx, y=yy,
                     label=' [Slope '+ str(np.round(mank[0], 3))+'] \n,[tau '+
               str(np.round(tau,3))+', pval '+str(np.round(pval,3))+']', s=2.5, color=ccolor, alpha=0.8)
        ax[idy, idx].plot(xx, xx*mank[0]+mank[1], color='black')
        ax[idy, idx].set_title(crop_name+' ('+hazard+')'+' :'+col, fontsize=15)
        ax[idy, idx].tick_params(axis='both',which='major',labelsize=14, pad=10)
        ax[idy, idx].legend(fontsize=14)
        ax[idy, idx].set_xlabel('Yield Loss (tons/ha)', fontsize=13) #Yield Loss (- 100 kg/ha)
        #ax.set_ylim(-5,9)
        ax[idy, idx].set_ylabel('Loss Cost ($)',fontsize=13)
        cor_temp = cor_temp.append(pd.DataFrame({'Hydro':hazard, 'Exposure': [exp], 'Cause of Loss': [col], 'Slope': np.round(mank[0],3), 
                           'Tau': np.round(tau,3), 'pval': np.round(pval,3), 
                                                 'rho':np.round(rho,3),'rho_pval':np.round(s_pval,3), 
                                                 'r':np.round(r,3), 'r_pval':np.round(r_pval,3)}))
#plt.savefig('fig/'+crop_name+'plot_revenue_based.png')

In [55]:
if crop == 'WHEAT':
    cor_temp['crop'] = crop_name
    cor_temp = cor_temp.loc[cor_temp['Cause of Loss'] != 'Rain/Flood'].copy()
else:
    cor_temp['crop'] = crop_name
cor_temp['cutoff_hazard'] = 15
cor_temp

,Hydro,Exposure,Cause of Loss,Slope,Tau,pval,rho,rho_pval,r,r_pval,crop,cutoff_hazard
0,HighD+LowS,10.841220,Heat,0.176,0.185,0.000,0.275,0.000,0.281,0.000,Spring Wheat,15
0,HighD+LowS,10.841220,Drought,2.504,0.380,0.000,0.542,0.000,0.565,0.000,Spring Wheat,15
0,HighD+LowS,10.841220,Disease,-0.128,-0.202,0.000,-0.297,0.000,-0.167,0.000,Spring Wheat,15
0,HighD+HighS,2.180177,Heat,0.473,0.109,0.000,0.158,0.000,0.051,0.225,Spring Wheat,15
0,HighD+HighS,2.180177,Drought,0.350,0.047,0.033,0.069,0.032,0.031,0.339,Spring Wheat,15
0,HighD+HighS,2.180177,Disease,-0.684,-0.121,0.001,-0.174,0.001,-0.147,0.004,Spring Wheat,15
0,LowD+LowS,0.257590,Heat,-0.126,-0.048,0.252,-0.068,0.237,-0.058,0.313,Spring Wheat,15
0,LowD+LowS,0.257590,Drought,-0.405,-0.024,0.482,-0.031,0.499,-0.022,0.628,Spring Wheat,15
0,LowD+LowS,0.257590,Disease,-0.116,-0.062,0.255,-0.082,0.269,0.107,0.149,Spring Wheat,15


In [271]:

cor_temp.to_csv('../3_outputs/Final_revenue_based_correlation_all_15%.csv')

In [57]:
cor_df = pd.concat([cor_df, cor_temp])


In [322]:
cor_df.to_csv('../3_outputs/Final_revenue_based_correlation_all_15%.csv')

In [3]:
## tau plot:

In [59]:
cor_df.loc[cor_df['Cause of Loss'] == 'Drought', 'xx'] = 0.8
cor_df.loc[cor_df['Cause of Loss'] == 'Heat', 'xx'] = 0.6
cor_df.loc[cor_df['Cause of Loss'] == 'Rain/Flood', 'xx'] = 0.4
cor_df.loc[cor_df['Cause of Loss'] == 'Disease', 'xx'] = 0.2


In [324]:
cor_df.loc[cor_df['crop'] == 'CORN', 'crop'] = 'Maize'
cor_df.loc[cor_df['crop'] == 'SOYBEANS', 'crop'] = 'Soybeans'
cor_df.loc[cor_df['crop'] == 'GRAIN SORGHUM', 'crop'] = 'Sorghum'
cor_df.loc[cor_df['crop'] == 'Spring Wheat', 'crop'] = 'S.Wheat'

cor_df.loc[cor_df['crop'] == 'Maize', 'crop_id'] = 1
cor_df.loc[cor_df['crop'] == 'Soybeans', 'crop_id'] = 2
cor_df.loc[cor_df['crop'] == 'Sorghum', 'crop_id'] = 3
cor_df.loc[cor_df['crop'] == 'S.Wheat', 'crop_id'] = 4

cor_df.loc[(cor_df['Exposure'] >= 10), 'ap_expo'] = 14
cor_df.loc[(cor_df['Exposure'] < 10) & (cor_df['Exposure'] >=5), 'ap_expo'] = 10.7
cor_df.loc[(cor_df['Exposure'] < 5) & (cor_df['Exposure'] >= 2), 'ap_expo'] = 4.7
cor_df.loc[cor_df['Exposure'] < 2 , 'ap_expo'] = 0.8

In [325]:
import matplotlib
matplotlib.rc('font', family='sans-serif') 
plt.rcParams.update({'font.sans-serif':'Arial'})
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [326]:
cor_df.loc[cor_df['Hydro'] == 'HighD+LowS', 'hydro_id'] = 11
cor_df.loc[cor_df['Hydro'] == 'LowD+HighS', 'hydro_id'] = 12
cor_df.loc[cor_df['Hydro'] == 'HighD+HighS', 'hydro_id'] = 13
cor_df.loc[cor_df['Hydro'] == 'LowD+LowS', 'hydro_id'] = 14

In [60]:
test = cor_df.loc[(cor_df['cutoff_hazard']==15)]
fig, ax = plt.subplots(figsize=(13,5), nrows=1, ncols=4, sharey=True)
ind=0
new_df = pd.DataFrame()
exp_size = sorted(cor_df['ap_expo'].unique())
exp_size[-1] = 18
legend_list = [Line2D([0], [0], marker='o',color='k', markersize=s+11, markerfacecolor='white') for s in exp_size]
labels = ['< 2', '2-5', '5-10', '>=10']

for key, grp in test.groupby(['hydro_id']):
    grp = grp.sort_values(by=['crop_id','xx'])
    new_df = new_df.append(grp)
    figg = ax[ind].scatter(x=grp['crop_id'], y=grp['xx'], s=grp['ap_expo'] *75, c=grp['Tau'], cmap='seismic',
              vmin=-0.5, vmax=0.5, edgecolor=['k' if x else  'lightgrey'  for x in (grp['r_pval'] <0.05)], label='size', linewidth=2)#, marker='s')
    ax[ind].set_frame_on(False)
    ax[ind].set_axisbelow(True) #grid lines are behind other objects

    ax[ind].xaxis.grid(which='major',color='black', b=False)
    ax[ind].yaxis.grid(which='major',color='black', b=False)

    ax[ind].set_ylim(0.15, 0.9)
    ax[ind].set_xlim(grp['crop_id'].min()-1, grp['crop_id'].max()+1)
    ax[ind].set_xticks(grp.sort_values('crop_id')['crop_id'].unique())
    ax[ind].set_xticklabels(grp.sort_values(by=['crop_id'])['crop'].drop_duplicates(), fontsize=22, rotation=90)
    ax[ind].tick_params(size=22) 
    ind=ind+1
    
ax[0].set_yticks(sorted(grp['xx'].unique()))
ax[0].set_ylabel('Crop Insurance\n Hazard Categories', fontsize=22)
ax[0].set_yticklabels(grp.sort_values('xx')['Cause of Loss'].unique(), fontsize=22)

cbar_ax = fig.add_axes([0.15, -0.55, 0.35, 0.07])
plt.figtext(0.18, -0.68, "Significance (p<0.05): Black Outline", ha="left", fontsize=17, color='black') 
cb= fig.colorbar(figg, orientation='horizontal', cax=cbar_ax)
cb.ax.tick_params(labelsize=18)
cb.ax.set_title('Correlation (\u03C4) between\nCrop Insurance Loss Cost ($)\nEstimated Yield Losses (tonnes/ha)', fontsize=20,pad=10)

leg = fig.legend(legend_list, labels, bbox_to_anchor=(0.7, -0.3, 0.1, 0.05), title='Hazard Exposure (%)',
           title_fontsize=18, frameon=False, fontsize=16, labelspacing = 1)
leg._legend_box.sep = 10

#plt.savefig('fig/final_total_based_liability_hydro_insurance_relation.pdf',dpi=100,bbox_inches='tight' )
#plt.savefig('fig/final_total_based_liability_hydro_insurance_relation.png',dpi=200,bbox_inches='tight' )